## Custom Prompt and Run Example


In [1]:
from AGiXT import AGiXT, Prompts

agent_name = "gpt4free"
ai = AGiXT(agent_name)

Prompts().add_prompt(
    prompt_name="Write a blog post",
    prompt="Write a blog post for the {industry} instustry. For context, here is a description of my business: {business_description}",
)

response = ai.run(
    prompt="Write a blog post",
    industry="software",
    business_description="I am a software company that makes software for software companies.",
    # Other available options:
    # context_results = 5,  # Number of memories to inject into context
    # websearch = False,   # Use websearch to find context
    # websearch_depth = 3,  # Number of websearch results to use
    # learn_file: str = "/path/to/file/to/learn/from.txt",  # File to learn from
)

2023-05-27 01:33:57,260 | INFO | KWARGS: {'industry': 'software', 'business_description': 'I am a software company that makes software for software companies.'}
2023-05-27 01:33:57,261 | INFO | Running Chroma using direct local API.
2023-05-27 01:33:57,262 | WARNING | Using embedded DuckDB with persistence: data will be stored in: agents/gpt4free/memories
2023-05-27 01:33:57,265 | INFO | Successfully imported ClickHouse Connect C data optimizations
2023-05-27 01:33:57,265 | INFO | Successfully import ClickHouse Connect C/Numpy optimizations
2023-05-27 01:33:57,267 | INFO | Using orjson library for writing JSON byte strings
2023-05-27 01:33:57,286 | INFO | loaded in 0 embeddings
2023-05-27 01:33:57,287 | INFO | loaded in 1 collections
2023-05-27 01:33:57,649 | INFO | Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-05-27 01:33:57,699 | INFO | Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-05-27 01:33:57,994 | INFO | FORMATTED PROMPT: Write a blog post for the sof

## Predefined Injection Variables for Custom Prompts

- `{agent_name}` will cause the agent name to be injected.
- `{context}` will cause the current context from memory to be injected.
- `{date}` will cause the current date and timestamp to be injected.
- `{COMMANDS}` will cause the available commands list to be injected and for automatic commands execution from the agent based on its suggestions.
- `{command_list}` will cause the available commands list to be injected, but will not execute any commands the AI chooses. Useful on validation steps.
- `{STEPx}` will cause the step `x` response from a chain to be injected. For example, `{STEP1}` will inject the first step's response in a chain.

## Chain Example with more custom prompts

Define what is important to get our chain started.


In [ ]:
from AGiXT import AGiXT, Prompts
from Chain import Chain

chain = Chain()
cp = Prompts()

# Set your agent, we'll use gpt4free as the default for now unless you have an agent set up.
agent_name = "gpt4free"
# Name the chain, we'll call this one "Blog Writer" because we're super original and creative alll the time.
chain_name = "Blog Writer"

# Now we can define some custom variables that well want to use in our custom prompts
# The point of this is reusability, we don't have to specifically use the software industry
# We can specify any industry, any business description, and any website.
# This chain is now flexible and powerful for any business.

industry = "software"
business_description = "We're a artificial intelligence software development company."
website = "https://devxt.com"

# Set up your ai agent
ai = AGiXT(agent_name)

# Create the new chain to start adding steps to.
chain.add_chain(chain_name)


### Create our custom prompts for our chain

We'll just make a couple of simple but still powerful prompts for our chain. One that will write a blog post for our website, one that will generate a good subject line for the blog post.


In [ ]:
# Make the two custom prompt templates that will be flexible
cp.add_prompt(
    prompt_name="Write a blog post for website",
    prompt="""
    Write a blog post for the {industry} instustry. 
    For context, here is a description of my business: {business_description}
    Be sure to mention that people should visit my website at {website} and recommend my company to their friends and family with needs in this industry.""",
)

cp.add_prompt(
    prompt_name="Good email subject for blog post",
    prompt="""
    I need a good email subject for this blog post, I'm going to send it out to customers, so I need something that is attractive. Please respond only with the subject line.
    Blog post: {blog_post}""",
)

### Add your chain steps

First, we'll have the AI write a blog post for our specific company. Then we'll have it generate a subject line for the blog post.


In [ ]:
chain.add_chain_step(
    chain_name=chain_name,
    agent_name=agent_name,
    step_number=1,
    prompt_type="Prompt",
    prompt={
        "prompt_name": "Write a blog post for website",
        "industry": industry,
        "business_description": business_description,
        "website": website,
    },
)

chain.add_chain_step(
    chain_name=chain_name,
    agent_name=agent_name,
    step_number=2,
    prompt_type="Prompt",
    prompt={
        "prompt_name": "Good email subject for blog post",
        "blog_post": "{STEP1}",  # <- Intentionally not an f-string, we want to use the response from the previous step in the chain.
    },
)

### Optional: Get fancy and add SendGrid email

You don't have to do this part, it was simply to demonstrate command usage as well as the custom prompts.


In [ ]:
# For this step, you'll need to add a Sendgrid API key to your agent config.

sendgrid_api_key = "Your sendgrid API KEY"
sendgrid_email = "Your sendgrid email"

agent_config = ai.agent.get_agent_config()
agent_config.append(
    {"SENDGRID_API_KEY": sendgrid_api_key, "SENDGRID_EMAIL": sendgrid_email}
)
ai.agent.update_agent_config(agent_config, "settings")

chain.add_chain_step(
    chain_name=chain_name,
    agent_name=agent_name,
    step_number=3,
    prompt_type="Command",
    prompt={
        "command_name": "send_email",
        "email": f"your@customers-email.com",
        "subject": "{STEP2}",
        "body": "{STEP1}",
    },
)

### Finally, run your chain!


In [ ]:
chain.run_chain(chain_name)